In [2]:
import pandas as pd
import pickle

### Same logic as Calculating_Distances.ipynb
* Calculating neighbor info for bating operations performed by NYC
* See associated markdown for info

In [3]:
with open("df_address.pkl", 'rb') as picklefile: 
    df4 = pickle.load(picklefile)

In [4]:
df = df4[['Block','XCoord','YCoord']].copy()
df['x'] = 1

In [5]:
df = df.groupby(['Block','XCoord','YCoord'], as_index=False).x.count()
del df['x']

In [6]:
df = df.groupby('Block')['XCoord','YCoord'].mean()

In [7]:
from scipy.spatial.distance import pdist, squareform

distances = pdist(df.values, metric='euclidean')
dist_matrix = squareform(distances)

In [8]:
indicies = ['block' + str(i) for i in df.index.tolist()]

In [9]:
dist_matrix = pd.DataFrame(dist_matrix, columns = indicies, index = df.index)

In [10]:
smallest = 6

m_len = len(dist_matrix.columns)
for col in dist_matrix.columns[:m_len]:
    l = dist_matrix[col].nsmallest(smallest).index
    for i in range(len(l)):
        col_2c = m_len + i
        row_2c = int(col[5:])
        dist_matrix.loc[row_2c, col_2c] = int(l[i])
        
k = 0
for j in range(m_len, m_len + smallest):
    k += 1
    dist_matrix.rename(columns={j: 'closest_' + str(k-1)}, inplace=True)

In [11]:
df_closest = dist_matrix.iloc[:,-smallest:].reset_index()

In [12]:
with open('df_closest.pkl', 'wb') as picklefile:
    pickle.dump(df_closest, picklefile)

In [13]:
with open("df_bait4.pkl", 'rb') as picklefile: 
    df_l4 = pickle.load(picklefile)

In [14]:
with open("df_all.pkl", 'rb') as picklefile: 
    df_all = pickle.load(picklefile)

In [15]:
df_l4 = df_all.merge(df_l4, how = 'left', on=['INSPECTION_YEAR_Q','BLOCK'])
df_l4.fillna(0, inplace=True)

In [16]:
for i in range(1,smallest):
    df_closest['closest_' + str(i)] = df_closest['closest_' + str(i)].astype(int)

In [17]:
df9 = df_l4.merge(df_closest, how = 'right', left_on='BLOCK', right_on='Block')
del df9['Block']

In [18]:
for i in range(1,smallest):
    df9 = df9.merge(df_l4, left_on=['INSPECTION_YEAR_Q','closest_' + str(i)], right_on=['INSPECTION_YEAR_Q','BLOCK'])
    del df9['BLOCK_y']
    df9.rename(columns={'last_4_bait_y': 'c' + str(i) + '_b4i','BLOCK_x': 'BLOCK',
                       'last_4_bait_x': 'last_4_bait'}, inplace=True)

In [19]:
col_to_keep = ['INSPECTION_YEAR_Q', 'BLOCK', 'last_4_bait',
               'c1_b4i', 'c2_b4i', 'c3_b4i', 'c4_b4i', 'c5_b4i']
df9 = df9[col_to_keep]

In [20]:
with open('last_4_bait_data.pkl', 'wb') as picklefile:
    pickle.dump(df9, picklefile)